<a href="https://colab.research.google.com/github/a-agmon/interviewdata/blob/main/Interview_Answers1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Please start by running the following cells that will download the data and the Spark environment.
#### Questions start after this part

In [1]:
!wget -q https://raw.githubusercontent.com/a-agmon/interviewdata/main/daily-transactions-2020-10-01
!wget -q https://raw.githubusercontent.com/a-agmon/interviewdata/main/daily-transactions-2020-10-02
!wget -q https://raw.githubusercontent.com/a-agmon/interviewdata/main/daily-transactions-2020-10-03

In [2]:
!rm -rf transactions-postproc
!rm -rf daily-transactions
!mkdir daily-transactions
!mv daily-*2020* daily-transactions/

In [3]:
!apt-get install tree
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (122 kB/s)
Selecting previously unselected package tree.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get

In [4]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 56.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2e7296b2585fcddf9b5edfa22d2843cc4e0802fbb3e3912418e4ec536478404f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

import pandas as pd
import numpy as np
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
conf = SparkConf()

Please also run these

In [74]:
packs =  [
    {'pack':1, 'pack_start_date':123456, 'pack_end_date':123460, 'pack_installs':10},
    {'pack':2, 'pack_start_date':123460, 'pack_end_date':123470, 'pack_installs':5},
    {'pack':3, 'pack_start_date':123456, 'pack_end_date':123458, 'pack_installs':10}]

consumption = [
    {'account':'AB','install_date':123459, 'installs':10},
    {'account':'AB','install_date':123465, 'installs':5},
    {'account':'AB','install_date':123466, 'installs':3}]

### * START HERE *

In [75]:
# run these and view the generated tables

packs_df = spark.createDataFrame(packs)
consumption_df = spark.createDataFrame(consumption)

print("Packages table\n")
packs_df.show()
print("consumption table\n")
consumption_df.show()

packs_df.createOrReplaceTempView("packs")
consumption_df.createOrReplaceTempView("consumption")

Packages table

+----+-------------+-------------+---------------+
|pack|pack_end_date|pack_installs|pack_start_date|
+----+-------------+-------------+---------------+
|   1|       123460|           10|         123456|
|   2|       123470|            5|         123460|
|   3|       123458|           10|         123456|
+----+-------------+-------------+---------------+

consumption table

+-------+------------+--------+
|account|install_date|installs|
+-------+------------+--------+
|     AB|      123459|      10|
|     AB|      123465|       5|
|     AB|      123466|       3|
+-------+------------+--------+



In [76]:
# an example to how we should run a spark query 
spark.sql("select * from packs").show()

+----+-------------+-------------+---------------+
|pack|pack_end_date|pack_installs|pack_start_date|
+----+-------------+-------------+---------------+
|   1|       123460|           10|         123456|
|   2|       123470|            5|         123460|
|   3|       123458|           10|         123456|
+----+-------------+-------------+---------------+



We would like to know how many installs users used from their packages in total and what is the difference from what their table allows


```
+-------+----+-------------+-------------+----+
|account|pack|pack_installs|sum(installs)|left|
+-------+----+-------------+-------------+----+
|     AB|   1|           10|           10|   0|
|     AB|   2|            5|            8|  -3|
+-------+----+-------------+-------------+----+

```

In [71]:
sqlQuery = """
  


"""

In [ ]:
spark.sql(sqlQuery).show()

A developer on the team wrote an ETL that runs once a day as a Saprk job.
Every day it reads a csv file that shows the total value of each customer's transactions of that day and write them as a parquet file partitioned by date and customer id.
Below you can see an example of the CSV file. Note that each customer has one entry that represents the total sum of transaction value it did on that day.

However, sometimes the csv file contains a correction for a sum reported in the past. 

for example - This file represents the transactions on the 1/10. You can see that **customer 1002** has 2 entries. One for the 1/10 and one for 30/9. This means that the total sum of transactions the customer did on the 1/10 is 70, but also that the total sum of transaction it did on the 30/9 was 40 and that this sum should **replace** the value already reported on the 30/9. 


```
current date file: 2020-10-01

date,customer,price
2020-10-01,1000,40
2020-10-01,1001,10
2020-09-30,1002,40
2020-10-01,1002,70
2020-10-01,1003,10
2020-09-29,1004,10
2020-10-01,1004,10
```

After the transformations files written in this partitioning scheme based on date and customer id

```
|_date=2000-1-1
|___customer=100
|_______file.p
|___customer=101

```


In [ ]:
# This is the folder that the prq files are written to
# before running the ETL this should be cleared 
!rm -rf transactions-postproc/

This function represents the ETL. It runs once a day with a string represening the current day. 

It reads the csv file, does some transformations, and write it.

In [ ]:
def run_etl(current_date): 

  df = spark.read.option("header",True).csv(f"daily-transactions/daily-transactions-{current_date}")
  
  df = df.withColumn("priceNumeric", F.col("price").astype(IntegerType()))
  
  # some other transformation code 

  df.write \
  .option("header",True) \
  .partitionBy("date") \
  .mode("overwrite") \
  .parquet("transactions-postproc")

This cell simulate the ETL running over 3 days for testing purposes

In [ ]:
%%time
# takes a minute to run!
days = ['2020-10-01', '2020-10-02', '2020-10-03']

for date_str in days:
  run_etl(date_str)

CPU times: user 107 ms, sys: 9.36 ms, total: 116 ms
Wall time: 14 s


Run the two lines below to test the results that should sum how much did the company made each day from all the customers

In [ ]:
df = spark.read.option("header",True).parquet("transactions-postproc")

In [ ]:
df.groupBy("date") \
.sum("priceNumeric") \
.sort("date") \
.show(10, False)

+----------+-----------------+
|date      |sum(priceNumeric)|
+----------+-----------------+
|2020-10-01|5120             |
|2020-10-02|5190             |
|2020-10-03|36610            |
+----------+-----------------+



Finance saw these results, and told us that there is an error here. They did the calculations manually and told us that it is supposed to be like this:


```

+----------+-----------------+
|date      |sum(priceNumeric)|
+----------+-----------------+
|2020-09-29|4880             |
|2020-09-30|9790             |
|2020-10-01|35330            |
|2020-10-02|32940            |
|2020-10-03|36610            |
+----------+-----------------+

```


Please help us find the bug in the code above, and return the right results

#### In another part of the job, the developer had to join the resul with dimentional table of categories. The join works, but its a bit slow, see if you can understand why and whether it can run faster

In [ ]:
ratesCategory = [('Small Money',10),('Some Money',20),('Nice Value',40),('BigMoney',70)]

categoriesDF = spark.createDataFrame(ratesCategory,['Category','Value'])

In [ ]:
categoriesDF.show(10, False)

+-----------+-----+
|Category   |Value|
+-----------+-----+
|Small Money|10   |
|Some Money |20   |
|Nice Value |40   |
|BigMoney   |70   |
+-----------+-----+



In [ ]:
%%time 

from pyspark.sql.functions import broadcast

bigDF = categoriesDF.join(broadcast(df), categoriesDF.Value == df.priceNumeric)
bigDF.show(10, False)

+-----------+-----+-----+------------+----------+--------+
|Category   |Value|price|priceNumeric|date      |customer|
+-----------+-----+-----+------------+----------+--------+
|Small Money|10   |10   |10          |2020-10-02|1571    |
|Small Money|10   |10   |10          |2020-09-30|1551    |
|Small Money|10   |10   |10          |2020-10-01|1710    |
|Small Money|10   |10   |10          |2020-10-02|1802    |
|Small Money|10   |10   |10          |2020-10-02|1379    |
|Small Money|10   |10   |10          |2020-10-01|1232    |
|Small Money|10   |10   |10          |2020-09-29|1049    |
|Small Money|10   |10   |10          |2020-10-02|1213    |
|Small Money|10   |10   |10          |2020-10-03|1042    |
|Small Money|10   |10   |10          |2020-09-29|1811    |
+-----------+-----+-----+------------+----------+--------+
only showing top 10 rows

CPU times: user 61.1 ms, sys: 4.45 ms, total: 65.5 ms
Wall time: 7.94 s


Finally, the developer was running the following code which supposed to run some transformation on each partition of the data. But he recieved this RuntimeError below. can you explain what does this error means? (dont try to fix it)

In [ ]:
def perPartitionOperation(elements):
  columns = ["dollarValue","date", "customer"]
  newDF = spark.createDataFrame(data=elements, schema = columns)
  return newDF

In [ ]:
df = df.rdd.mapPartitions(perPartitionOperation)

RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers.

In [ ]:
df

Traceback (most recent call last):
  File "/content/spark-3.3.0-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/content/spark-3.3.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 102, in dumps
    cp.dump(obj)
  File "/content/spark-3.3.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/usr/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/usr/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python3.7/pickle.py", line 789, in save_tuple
    save(element)
  File "/usr/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/content/spark-3.3.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 784, in save_function
    *self._dynamic_function_reduce(obj), obj=obj


PicklingError: ignored